<a href="https://colab.research.google.com/github/shah-zeb-naveed/autogen-udemy-course/blob/main/ollama_on_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install litellm[proxy] aiohttp pyngrok -q

In [2]:
!ngrok config add-authtoken 2fKl7tkDAvFhoWbPRnkEOkZxXoy_3TP2baNeaJosAsZs838Ua

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10044    0 10044    0     0  45267      0 --:--:-- --:--:-- --:--:-- 45447
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [4]:
import threading
import time
import os
import asyncio
from pyngrok import ngrok
import threading
import queue
import time
from threading import Thread

# Get your ngrok token from your ngrok account:
# https://dashboard.ngrok.com/get-started/your-authtoken
token="2fKl7tkDAvFhoWbPRnkEOkZxXoy_3TP2baNeaJosAsZs838Ua"
ngrok.set_auth_token(token)

# set up a stoppable thread (not mandatory, but cleaner if you want to stop this later
class StoppableThread(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(StoppableThread, self).__init__(*args, **kwargs)
        self._stop_event = threading.Event()

    def stop(self):
        self._stop_event.set()

    def is_stopped(self):
        return self._stop_event.is_set()

def start_ngrok(q, stop_event):
    try:
        # Start an HTTP tunnel on the specified port
        public_url = ngrok.connect(11434)
        # Put the public URL in the queue
        q.put(public_url)
        # Keep the thread alive until stop event is set
        while not stop_event.is_set():
            time.sleep(1)  # Adjust sleep time as needed
    except Exception as e:
        print(f"Error in start_ngrok: {e}")

In [5]:
# Create a queue to share data between threads
url_queue = queue.Queue()

# Start ngrok in a separate thread
ngrok_thread = StoppableThread(target=start_ngrok, args=(url_queue, StoppableThread.is_stopped))
ngrok_thread.start()

In [6]:
# Wait for the ngrok tunnel to be established
while True:
    try:
        public_url = url_queue.get()
        if public_url:
            break
        print("Waiting for ngrok URL...")
        time.sleep(1)
    except Exception as e:
        print(f"Error in retrieving ngrok URL: {e}")

print("Ngrok tunnel established at:", public_url)

Error in start_ngrok: 'function' object has no attribute 'is_set'
Ngrok tunnel established at: NgrokTunnel: "https://fe57-104-197-158-196.ngrok-free.app" -> "http://localhost:11434"


In [7]:
import os
import asyncio

# NB: You may need to set these depending and get cuda working depending which backend you are running.
# Set environment variable for NVIDIA library
# Set environment variables for CUDA
os.environ['PATH'] += ':/usr/local/cuda/bin'
# Set LD_LIBRARY_PATH to include both /usr/lib64-nvidia and CUDA lib directories
os.environ['LD_LIBRARY_PATH'] = '/usr/lib64-nvidia:/usr/local/cuda/lib64'

async def run_process(cmd):
    print('>>> starting', *cmd)
    process = await asyncio.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )

    # define an async pipe function
    async def pipe(lines):
        async for line in lines:
            print(line.decode().strip())

        await asyncio.gather(
            pipe(process.stdout),
            pipe(process.stderr),
        )

    # call it
    await asyncio.gather(pipe(process.stdout), pipe(process.stderr))

In [9]:
import asyncio
import threading

async def start_ollama_serve():
    await run_process(['ollama', 'serve'])

def run_async_in_thread(loop, coro):
    asyncio.set_event_loop(loop)
    loop.run_until_complete(coro)
    loop.close()

# Create a new event loop that will run in a new thread
new_loop = asyncio.new_event_loop()

# Start ollama serve in a separate thread so the cell won't block execution
thread = threading.Thread(target=run_async_in_thread, args=(new_loop, start_ollama_serve()))
thread.start()

>>> starting ollama serve
Error: listen tcp 127.0.0.1:11434: bind: address already in use


In [18]:
!ollama pull tinyllama

[GIN] 2024/04/19 - 21:04:55 | 200 |      58.754µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ time=2024-04-19T21:04:57.123Z level=INFO source=download.go:136 msg="downloading 2af3b81862c6 in 7 100 MB part(s)"
pulling manifest ⠴ pulling manifest 
pulling 2af3b81862c6...   0% ▕▏    0 B/637 MB                  pulling manifest 
pulling 2af3b81862c6...   0% ▕▏    0 B/637 MB                  pulling manifest 
pulling 2af3b81862c6...   0% ▕▏    0 B/637 MB                  pulling manifest 
pulling 2af3b81862c6...   0% ▕▏    0 B/637 MB                  pulling manifest 
pulling 2af3b81862c6...   0% ▕▏    0 B/637 MB                  pulling manifest 
pulling 2af3b81862c6...   0% ▕▏    0 B/637 MB                  pulling mani

In [19]:
!ollama run tinyllama

[GIN] 2024/04/19 - 21:06:22 | 200 |      23.893µs |       127.0.0.1 | HEAD     "/"
[GIN] 2024/04/19 - 21:06:22 | 200 |     761.792µs |       127.0.0.1 | POST     "/api/show"
[GIN] 2024/04/19 - 21:06:22 | 200 |     421.239µs |       127.0.0.1 | POST     "/api/show"
⠙ ⠹ time=2024-04-19T21:06:23.306Z level=INFO source=gpu.go:121 msg="Detecting GPU type"
time=2024-04-19T21:06:23.306Z level=INFO source=gpu.go:268 msg="Searching for GPU management library libcudart.so*"
time=2024-04-19T21:06:23.312Z level=INFO source=gpu.go:314 msg="Discovered GPU libraries: [/tmp/ollama297297673/runners/cuda_v11/libcudart.so.11.0 /usr/local/cuda/lib64/libcudart.so.12.2.140]"
time=2024-04-19T21:06:23.313Z level=INFO source=gpu.go:126 msg="Nvidia GPU detected via cudart"
time=2024-04-19T21:06:23.313Z level=INFO source=cpu_common.go:11 msg="CPU has AVX2"
⠸ ⠼ time=2024-04-19T21:06:23.520Z level=INFO source=gpu.go:202 msg="[cudart] CUDART CUDA Compute Capability detected: 7.5"
time=2024-04-19T21:06:23.547Z level

In [14]:
!pip install pyautogen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.9/264.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 23.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.23.2
    Uninstalling openai-1.23.2:
      Successfully uninstalled openai-1.23.2


In [11]:
!python -m litellm --model ollama_chat/tinyllama

/usr/bin/python3: No module named litellm.__main__; 'litellm' is a package and cannot be directly executed


In [1]:
!python -m litellm --model ollama_chat/tinyllama

/usr/bin/python3: No module named litellm.__main__; 'litellm' is a package and cannot be directly executed


In [2]:
!python -m litellm.__main__ --model ollama_chat/tinyllama

/usr/bin/python3: No module named litellm.__main__


In [1]:
!pip install vllm -q

  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1+cu121
    Uninstalling torch-2.2.1+cu121:
      Successfully uninstalled torch-2.2.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.1.2 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.1.2 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.1.2 which is incompatible.


In [1]:
import os
from google.colab import userdata

os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HF_TOKEN')

In [ ]:
!python -m vllm.entrypoints.openai.api_server --max-model-len 20 --gpu-memory-utilization 0.9 --model meta-llama/Llama-2-7b-chat-hf --chat-template chat_template.jinja

In [ ]:
self.max_tokens

In [6]:
import subprocess
import time

# Define the command to start the service
service_command = [
    'python', '-m', 'vllm.entrypoints.openai.api_server',
    #'--max-model-len', '200',
    '--gpu-memory-utilization', '0.9',
    '--model', 'meta-llama/Llama-2-7b-chat-hf',
    '--chat-template', 'chat_template.jinja'
]

# Start the service in a separate process and capture its output
service_process = subprocess.Popen(service_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Poll the output to check if the service is up and running
while True:
    output_line = service_process.stderr.readline()
    if "Uvicorn running on" in output_line:
        print("Service is up and running!")
        break
    elif output_line == '' and service_process.poll() is not None:
        print("Failed to start the service.")
        break

# Code to consume the service
from autogen import UserProxyAgent, ConversableAgent

local_llm_config={
    "config_list": [
        {
            "model": "meta-llama/Llama-2-7b-chat-hf", # Same as in vLLM command
            "api_key": "NotRequired", # Not needed
            "base_url": "http://0.0.0.0:8000/v1"  # Your vLLM URL, with '/v1' added
        }
    ],
    "cache_seed": None # Turns off caching, useful for testing different models
}

# Create the agent that uses the LLM.
assistant = ConversableAgent("agent", llm_config=local_llm_config,system_message="")

# Create the agent that represents the user in the conversation.
user_proxy = UserProxyAgent("user", code_execution_config=False,system_message="")

# Let the assistant start the conversation.  It will end when the user types exit.
assistant.initiate_chat(user_proxy, message="How can I help you today?")


Failed to start the service.
agent (to user):

How can I help you today?

--------------------------------------------------------------------------------
Provide feedback to agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: hi
user (to agent):

hi

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...


BadRequestError: Error code: 400 - {'object': 'error', 'message': 'max_tokens must be at least 1, got -18.', 'type': 'BadRequestError', 'param': None, 'code': 400}

In [ ]:
from autogen import UserProxyAgent, ConversableAgent

local_llm_config={
    "config_list": [
        {
            "model": "mistralai/Mistral-7B-Instruct-v0.2", # Same as in vLLM command
            "api_key": "NotRequired", # Not needed
            "base_url": "http://0.0.0.0:8000/v1"  # Your vLLM URL, with '/v1' added
        }
    ],
    "cache_seed": None # Turns off caching, useful for testing different models
}

# Create the agent that uses the LLM.
assistant = ConversableAgent("agent", llm_config=local_llm_config,system_message="")

# Create the agent that represents the user in the conversation.
user_proxy = UserProxyAgent("user", code_execution_config=False,system_message="")

# Let the assistant start the conversation.  It will end when the user types exit.
assistant.initiate_chat(user_proxy, message="How can I help you today?")